In [2]:
# Josh Kreutz
# 11/21/2024
# AMATH 581 HW 5

# Step 1: Construct matrix A from HW 4
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import spdiags
from scipy.sparse import spdiags, block_diag
from scipy.integrate import solve_ivp  
from scipy.linalg import lu_factor, lu_solve
import time

grid_size = 64    
total_points = grid_size * grid_size  

zero_vector = np.zeros((total_points, 1)) 
one_vector = np.ones((total_points, 1))   
modified_one_vector = np.copy(one_vector)    
boundary_condition_vector = np.copy(zero_vector)    

# Update boundary conditions
for row in range(1, grid_size + 1):
    modified_one_vector[grid_size * row - 1] = 0  
    boundary_condition_vector[grid_size * row - 1] = 1  

# Shift the modified vectors to form e3 and e5
e3 = np.zeros_like(modified_one_vector)
e3[1:total_points] = modified_one_vector[0:total_points - 1]
e3[0] = modified_one_vector[total_points - 1]

e5 = np.zeros_like(boundary_condition_vector)
e5[1:total_points] = boundary_condition_vector[0:total_points - 1]
e5[0] = boundary_condition_vector[total_points - 1]

# Construct the sparse diagonals
diagonals = [one_vector.flatten(), one_vector.flatten(), e5.flatten(), 
             modified_one_vector.flatten(), -4 * one_vector.flatten(), e3.flatten(), 
             boundary_condition_vector.flatten(), one_vector.flatten(), one_vector.flatten()]

# Set the offsets for the sparse matrix
offsets = [-(total_points - grid_size), -grid_size, -grid_size + 1, -1, 0, 1, grid_size - 1, grid_size, (total_points - grid_size)]

# Create matrix A using the sparse diagonals and offsets
matrix_A = spdiags(diagonals, offsets, total_points, total_points).toarray()

A = matrix_A

# Step 2: Construct matrix B from HW 4
def create_expanded_matrix(n, identity_size=64):
    # Create a small tridiagonal matrix
    data = np.zeros((3, n))
    data[0, :] = 0     
    data[1, 1:] = 1    
    data[2, :-1] = -1  
    
    diagonals = np.array([0, 1, -1])
    small_matrix = spdiags(data, diagonals, n, n).toarray()

    # Apply boundary conditions
    small_matrix[0, n - 1] = -1  
    small_matrix[n - 1, 0] = 1   

    # Create identity matrices
    identity_matrix = np.eye(identity_size)
    negative_identity_matrix = -np.eye(identity_size)
    zero_matrix = np.zeros((identity_size, identity_size))

    # Initialize the expanded matrix
    expanded_matrix = np.zeros((n * identity_size, n * identity_size))

    # Fill the expanded matrix with identity and negative identity blocks
    for i in range(n):
        for j in range(n):
            if small_matrix[i, j] == 1:
                expanded_matrix[i * identity_size:(i + 1) * identity_size, 
                                j * identity_size:(j + 1) * identity_size] = identity_matrix
            elif small_matrix[i, j] == -1:
                expanded_matrix[i * identity_size:(i + 1) * identity_size, 
                                j * identity_size:(j + 1) * identity_size] = negative_identity_matrix
            else:
                expanded_matrix[i * identity_size:(i + 1) * identity_size, 
                                j * identity_size:(j + 1) * identity_size] = zero_matrix

    return expanded_matrix

expanded_matrix = create_expanded_matrix(64)
B = expanded_matrix

# Step 3: Construct matrix C from HW 4
from scipy.sparse import block_diag

def create_matrix(n):
    data = np.zeros((3, n))  
    data[0, :] = 0  
    data[1, 1:] = 1  
    data[2, :-1] = -1  
    
    diagonals = np.array([0, 1, -1])
    small_matrix = spdiags(data, diagonals, n, n).toarray()
    
    small_matrix[0, n - 1] = -1  
    small_matrix[n - 1, 0] = 1   
    
    return small_matrix

small_matrix_C = create_matrix(64)
num_blocks = 64  
C = block_diag([small_matrix_C] * num_blocks).toarray()

# Part A: Step 4: Fourier Transform setup
import numpy as np 
import matplotlib.pyplot as plt
from scipy.fftpack import fft2, ifft2
import time

viscosity = 0.001
Lx, Ly = 20, 20
nx, ny = 64, 64
num_points = nx * ny
time_points = np.arange(0, 4.5, 0.5)

# Create mesh grid and initial condition
x_vals = np.linspace(-Lx / 2, Lx / 2, nx + 1) 
y_vals = np.linspace(-Ly / 2, Ly / 2, ny + 1) 
x = x_vals[:nx]
y = y_vals[:ny]
X, Y = np.meshgrid(x, y)
initial_condition = np.exp(-X ** 2 - Y ** 2 / 20)

initial_condition = initial_condition.flatten()
A = (1 / (x[1] - x[0]) ** 2) * A
B = (1 / (2 * (x[1] - x[0]))) * B
C = (1 / (2 * (x[1] - x[0]))) * C

# Define wave numbers for FFT
kx = (2 * np.pi / Lx) * np.concatenate((np.arange(0, nx / 2), np.arange(-nx / 2, 0)))
kx[0] = 1e-6
ky = (2 * np.pi / Ly) * np.concatenate((np.arange(0, ny / 2), np.arange(-ny / 2, 0)))
ky[0] = 1e-6
KX, KY = np.meshgrid(kx, ky)
K = KX ** 2 + KY ** 2

# Step 5: Solve ODE with FFT
def rhs_fft(t, w, K, nu):
    w_matrix = w.reshape((nx, ny))  
    w_fft = fft2(w_matrix) 
    psi_fft = ifft2(-w_fft / K)  
    psi = np.real(psi_fft)
    psi = psi.flatten() 
    w = w.flatten() 
    
    rhs_val = nu * np.dot(A, w) - np.dot(B, psi) * np.dot(C, w) + np.dot(B, w) * np.dot(C, psi)
    return rhs_val

# Solve using ODE solver
start_time = time.time()
solution_fft = solve_ivp(rhs_fft, (time_points[0], time_points[-1]), initial_condition, method="RK45", t_eval=time_points, args=(K, viscosity))
end_time = time.time()
print(f'Execution time: {end_time - start_time:.2f} seconds')

solution_fft_result = solution_fft.y

print(solution_fft_result)

A1 = solution_fft_result

# AMINATION

# fig, ax = plt.subplots(figsize=(6, 6))
# cax = ax.imshow(A1[:, 0].reshape((nx, ny)), extent=[-10, 10, -10, 10], cmap='jet')
# fig.colorbar(cax, ax=ax, label='vorticity')
# ax.set_xlabel("x")
# ax.set_ylabel("y")
# def update(frame):
#     ax.set_title(f"Vorticity at t={frame*0.5:.2f}")
#     cax.set_data(A1[:,frame].reshape((nx,ny)))
#     return cax,
# anime=FuncAnimation(fig,update,frames=A1.shape[1],blit=True)
# anime.save("./vorti_direct.gif",writer="imagemagick",fps=2)
print(A1)


# Part B: Step 6: Modify matrix A for A/b method
modified_A = matrix_A
modified_A[0, 0] = 2
A = (1 / (x[1] - x[0]) ** 2) * modified_A

# A/backb method
def Abackb(t, w, nu=0.001):
    psi = np.linalg.solve(A, w)
    psi = psi.flatten()
    rhs_val = nu * np.dot(A, w) - np.dot(B, psi) * np.dot(C, w) + np.dot(B, w) * np.dot(C, psi)
    return rhs_val

start_time_Abackb = time.time()
solution_Abackb = solve_ivp(Abackb, (time_points[0], time_points[-1]), initial_condition, method="RK45", t_eval=time_points)
end_time_Abackb = time.time()
print(f'Execution time: {end_time_Abackb - start_time_Abackb:.2f} seconds')

solution_Abackb_result = solution_Abackb.y

print(solution_Abackb_result)

A2 = solution_Abackb_result

# AMINATION
# fig, ax = plt.subplots(figsize=(6, 6))
# cax = ax.imshow(A2[:, 0].reshape((nx, ny)), extent=[-10, 10, -10, 10], cmap='jet')
# fig.colorbar(cax, ax=ax, label='vorticity')
# ax.set_xlabel("x")
# ax.set_ylabel("y")
# def update(frame):
#     ax.set_title(f"Vorticity at t={frame*0.5:.2f}")
#     cax.set_data(A2[:,frame].reshape((nx,ny)))
#     return cax,
# anime=FuncAnimation(fig,update,frames=A2.shape[1],blit=True)
# anime.save("./vorti_Ab.gif",writer="imagemagick",fps=2)


# Part B: Step 7: LU decomposition method
from scipy.linalg import lu_factor, lu_solve

def lu_method(t, w, nu=0.001):
    LU, piv = lu_factor(A)
    psi = lu_solve((LU, piv), w)
    psi = psi.flatten()
    rhs_val = nu * np.dot(A, w) - np.dot(B, psi) * np.dot(C, w) + np.dot(B, w) * np.dot(C, psi)
    return rhs_val

start_time_LU = time.time()
solution_LU = solve_ivp(lu_method, (time_points[0], time_points[-1]), initial_condition, method="RK45", t_eval=time_points)
end_time_LU = time.time()
print(f'Execution time: {end_time_LU - start_time_LU:.2f} seconds')

solution_LU_result = solution_LU.y

print(solution_LU_result)

A3 = solution_LU_result

# AMINATION
# fig, ax = plt.subplots(figsize=(6, 6))
# cax = ax.imshow(A3[:, 0].reshape((nx, ny)), extent=[-10, 10, -10, 10], cmap='jet')
# fig.colorbar(cax, ax=ax, label='vorticity')
# ax.set_xlabel("x")
# ax.set_ylabel("y")
# def update(frame):
#     ax.set_title(f"Vorticity at t={frame*0.5:.2f}")
#     cax.set_data(A3[:,frame].reshape((nx,ny)))
#     return cax,
# anime=FuncAnimation(fig,update,frames=A3.shape[1],blit=True)
# anime.save("./vorti_lu.gif",writer="imagemagick",fps=2)


# Step 8: GMRES method
# from scipy.sparse.linalg import gmres as scipy_gmres
# import numpy as np
# from scipy.sparse import csr_matrix
# def gmres(t, w, nu=0.001):
   
#     A_sparse = csr_matrix(A)
#     psi, info = scipy_gmres(A_sparse, w, rtol=1e-6, maxiter=1000)  
#     if info != 0:
#         print(f"Warning: GMRES did not converge (info={info})")
   
#     psi = np.array(psi).flatten()
#     rhs_val = nu * np.dot(A, w) - np.dot(B, psi) * np.dot(C, w) + np.dot(B, w) * np.dot(C, psi)
#     return rhs_val
# sgm_time = time.time()
# wtsolgmres = solve_ivp(gmres, (tpan[0], tpan[-1]), w0, method="RK45", t_eval=tpan)
# egm_time=time.time()
# print(f'escape time:{egm_time-sgm_time:.2f}seconds')


# A4=wtsolgmres.y
# print(f"A4:{A4}")

# Step 9: BICGSTAB method
# from scipy.sparse.linalg import bicgstab
# def bicgstag(t, w, nu=0.001):
#     A_sparse = csr_matrix(A)  # Convert A to sparse matrix for BiCGSTAB
#     psi, info = bicgstab(A_sparse, w)
#     if info == 0:
#         psi = np.array(psi).flatten()
#         rhs_val = nu * np.dot(A, w) - np.dot(B, psi) * np.dot(C, w) + np.dot(B, w) * np.dot(C, psi)
#         return rhs_val
       
#     else:
#         raise Exception("BiCGSTAB did not converge")
    
# sbg_time = time.time()
# wtsolbg = solve_ivp(bicgstag, (tpan[0], tpan[-1]), w0, method="RK45", t_eval=tpan)
# ebg_time=time.time()
# print(f'escape time:{ebg_time-sbg_time:.2f}seconds')  

# A5=wtsolbg.y
# print(f"A5:{A5}")

# Part C Step 10 FFT
w2 = np.exp(-X**2 - Y**2/20)
w1 = np.exp(-(X**2)/20 - Y**2)
w3=w1+w2
w3=w3.flatten()
s2_time = time.time()
wtsol = solve_ivp(rhs_fft, (time_points[0], time_points[-1]), w3, method="RK45", t_eval=time_points, args=(K, viscosity))
e2_time=time.time()
print(f'escape time:{e2_time-s2_time:.2f}seconds')

plt.figure(figsize=(12, 8))

A6=wtsol.y
print(A6)

Execution time: 0.80 seconds
[[2.50656748e-46 3.54893701e-45 1.85736078e-44 ... 1.85335184e-42
  4.67274338e-42 1.10902720e-41]
 [1.17762859e-43 6.53684666e-43 2.62060667e-42 ... 1.58866337e-40
  3.55915506e-40 7.45532026e-40]
 [4.55107657e-41 1.93087628e-40 6.43663355e-40 ... 2.48966551e-38
  5.07459635e-38 9.69185374e-38]
 ...
 [1.96785570e-38 1.23636238e-37 5.46906656e-37 ... 4.35977114e-35
  1.02335874e-34 2.22858322e-34]
 [6.19028421e-41 5.34124949e-40 2.93882031e-39 ... 3.68664229e-37
  9.45501433e-37 2.23512843e-36]
 [1.60178709e-43 1.99879166e-42 1.41268009e-41 ... 2.80496038e-39
  7.96285821e-39 2.07493092e-38]]
[[2.50656748e-46 3.54893701e-45 1.85736078e-44 ... 1.85335184e-42
  4.67274338e-42 1.10902720e-41]
 [1.17762859e-43 6.53684666e-43 2.62060667e-42 ... 1.58866337e-40
  3.55915506e-40 7.45532026e-40]
 [4.55107657e-41 1.93087628e-40 6.43663355e-40 ... 2.48966551e-38
  5.07459635e-38 9.69185374e-38]
 ...
 [1.96785570e-38 1.23636238e-37 5.46906656e-37 ... 4.35977114e-35
  1

<Figure size 1200x800 with 0 Axes>